In [1]:
import socket
import binascii

from resolver.packet import DnsMessage, DnsQuestion, DnsResourceRecord
from resolver.record_type import QType, QClass
from resolver.utility import to_qname
from resolver.buffer import ByteBuffer

### Creating and receiving custom DNS datagrams with resolver.packet
#### resolver module introduces ORM-like layer for creating, parsing, sending and receiving DNS datagrams

In [2]:
# Empty message is created with default DNS datagram header
msg = DnsMessage()
msg

DNS Message: DnsHeader(ID=43690, response=False, opcode=0, authoritative_answer=False, truncation=False, recursion_desired=True, recursion_available=False, Z=2, response_code=<RCode.NO_ERROR: 0>, qdcount=0, ancount=0, nscount=0, arcount=0)

In [3]:
# Header can be accessed and modified
msg.header.recursion_desired = False
msg.header

DnsHeader(ID=43690, response=False, opcode=0, authoritative_answer=False, truncation=False, recursion_desired=False, recursion_available=False, Z=2, response_code=<RCode.NO_ERROR: 0>, qdcount=0, ancount=0, nscount=0, arcount=0)

In [4]:
# To add a question to the datagram instantiate DNSQuestion with desired parameters and add it to the message
msg.add_question(
    DnsQuestion(name="cs.berkeley.edu", qtype=QType.AAAA)
)
msg.question

[cs.berkeley.edu.: type: QType.AAAA, class: QClass.IN]

In [5]:
# This system supports OPT pseudo records for accepting DNS datagrams larger than 512 bytes in size
msg.add_pseudo_record(udp_payload_size=4096)
msg.additional

[.: type: QType.OPT, class: 4096 data: ]

In [6]:
# Any DNS resource record can also be added in similar way
# First prepare RDATA payload:
ns_rdata = "m.edu-servers.net."
encoded_domain_name = to_qname(domain_name=ns_rdata)
encoded_domain_name

'016d0b6564752d73657276657273036e657400'

In [7]:
from resolver.record_type import RData

# Prepare DnsResourceRecord instance
ns_resource = DnsResourceRecord(
    name="edu",
    qtype=QType.NS,
    qclass=QClass.IN,
    ttl=172800,
    rdata=RData(data=encoded_domain_name),
    rdlength=(len(encoded_domain_name) // 2)
)

In [8]:
# Add resource record to the message
msg.add_resource_record(ns_resource, section="answer")
msg.answer

[edu: type: QType.NS, class: QClass.IN data: 016d0b6564752d73657276657273036e657400]

In [9]:
# QName data can be translated back to human-readable format with ByteBuffer which provides a way for parsing encoded qnames:
buffer = ByteBuffer(binascii.unhexlify(ns_resource.rdata.data))
name = buffer.read_qname()
name

'm.edu-servers.net'

In [10]:
# The entire datagram can be viewed at any point in time:
msg.print_concise_info()

<<HEADER>> opcode: 0, status: RCode.NO_ERROR, id: 43690
flags: #query: 1, #answer: 1, #authority: 0, #additional: 1
<<QUESTION>>
cs.berkeley.edu.                                      IN    AAAA

<<ANSWER>>
edu.                                      172800      IN      NS        016d0b6564752d73657276657273036e657400

<<AUTHORITY>>

<<ADDITIONAL>>
..                                             0    4096     OPT        



In [11]:
# We will remove NS record added in answer section as it would result in a response with server failure flag. We will also set flag RD=True to get an answer
msg.answer = []
msg.header.ancount = 0
msg.header.recursion_desired = True

In [12]:
# Finally, DNSMessage can be translated into DNS datagram
payload = msg.build_bytes()
payload

b'\xaa\xaa\x01 \x00\x01\x00\x00\x00\x00\x00\x01\x02cs\x08berkeley\x03edu\x00\x00\x1c\x00\x01\x00\x00)\x10\x00\x00\x00\x00\x00\x00\x00'

In [13]:
# And sent with UDP to specified DNS server
server = ("1.1.1.1", 53)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
try:
    sock.sendto(msg.build_bytes(), server)
    data, _ = sock.recvfrom(4096)
finally:
    sock.close()

In [14]:
# The result is another DNS Datagram
data

b'\xaa\xaa\x81\x80\x00\x01\x00\x01\x00\x00\x00\x01\x02cs\x08berkeley\x03edu\x00\x00\x1c\x00\x01\xc0\x0c\x00\x1c\x00\x01\x00\x01Q\x80\x00\x10& \x01*\x80\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00)\x04\xd0\x00\x00\x00\x00\x00\x00'

In [15]:
# Which can be parsed to get results back
response = DnsMessage().from_bytes(data)
response.print_concise_info()

<<HEADER>> opcode: 0, status: RCode.NO_ERROR, id: 43690
flags: RD RA #query: 1, #answer: 1, #authority: 0, #additional: 1
<<QUESTION>>
cs.berkeley.edu.                                      IN    AAAA

<<ANSWER>>
cs.berkeley.edu.                           86400      IN    AAAA        2620:12a:8001::1

<<AUTHORITY>>

<<ADDITIONAL>>
.                                              0    1232     OPT        



In [16]:
# Class and method layout allows for chaining to facilitate a fluent-like API
datagram_plain = DnsMessage().add_question(
    DnsQuestion("yahoo.com")
).add_pseudo_record(udp_payload_size=1028).build()
datagram_plain

'aaaa01200002000000000002057961686f6f03636f6d00000100010000290404000000000000'

#### Resolver module can be extended to process any resource record or pseudo record
#### As it can both create, parse and manipulate DNS datagrams it could be used as a full-fledged DNS server when implemented with record caching mechanism